This is the main Sensor Kennel application that will 
identify, read and write any sensor that's plugged in.

On the lower side:  
  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.
  
Do we want to draw a graph of numbers if a button is pressed?

Should we have MQTT numbers going out from it?

Or just keep it comprehensive

In [787]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [791]:
print(i2c.readfrom_mem(0x77, 0xD0, 1)[0], 0x60)


96 96


In [209]:
%sendtofile --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source SDOF_funcs.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --binary --source OLED_driver.py
%sendtofile --quiet --source BNO055serial_funcs.py

Sent 55 lines (2042 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 42 lines (1800 bytes) to SDOF_funcs.py.
Sent 55 lines (1614 bytes) to device_detection.py.
Sent 3084 bytes in 103 chunks to OLED_driver.py.
Sent 88 lines (2732 bytes) to BNO055serial_funcs.py.


In [553]:
%sendtofile --binary --source OLED_driver.py


Sent 3083 bytes in 103 chunks to OLED_driver.py.


In [558]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.3-548-gd12483d9 on 2018-04-18; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.3-548-gd12483d9 on 2018-04-18; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [554]:
import os
os.remove("test.py")


In [559]:
import dmain
#dmain.fbuff
print(dmain.uart)


Temperature 24
.............................................
**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


**[ys] <class 'serial.serialutil.SerialException'>
**[ys] read failed: device reports readiness to read but returned no data (device disconnected or multiple access on port?)



In [522]:
import OLED_driver
print(dir(OLED_driver))
print(OLED_driver.i2c.scan(), 0x3C)
assert 0x3c in OLED_driver.i2c.scan()

['__class__', '__file__', '__name__', 'I2C', 'Pin', 'i2c', 'rst']
[60] 60


In [784]:
%sendtofile main.py

# connect to the OLED and initialize (and get its i2c bus)
from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext
import machine, time
time.sleep(0.5)

from device_detection import IdentifyI2CDevice, IdentifyUARTDevice

devices = IdentifyI2CDevice(i2c)
if not devices:
    uart = machine.UART(1, baudrate=9600, rx=13, tx=17)
    devices = IdentifyUARTDevice(uart)
if not devices:
    devices.append("nothing found")

# this might look like a lot of code, but it's the complete 
# visual output of text and numbers that we do per sensor here
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname, 0, 0, 1)
    fbuff.text(devdesc, 0, 12, 1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist: %.1f"%d, 8, 20, 1)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()
        time.sleep(0.1)

if devname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()
        time.sleep(0.1)

if devname == "SDOF":
    from SDOF_funcs import SetupAccGyrMag, readvectorsensor
    SetupAccGyrMag(i2c)
    while True:
        fbuff.fill(0)
        x, y, z = readvectorsensor("a")
        fbuff.text("Acc:", 0, 0, 1)
        fbuff.text("%d"%x, 8, 8, 1)
        fbuff.text("%d"%y, 8, 16, 1)
        fbuff.text("%d"%z, 8, 24, 1)
        x, y, z = readvectorsensor("g")
        fbuff.text("gyr:", 64, 0, 1)
        fbuff.text("%d"%x, 72, 8, 1)
        fbuff.text("%d"%y, 72, 16, 1)
        fbuff.text("%d"%z, 72, 24, 1)
        x, y, z = readvectorsensor("c")
        fbuff.text("mag:", 40, 32, 1)
        fbuff.text("%d"%x, 48, 40, 1)
        fbuff.text("%d"%y, 48, 48, 1)
        fbuff.text("%d"%z, 48, 56, 1)
        oledshow()
        time.sleep(0.1)
        
if devname == "BME280":
    from BME280_funcs import bme280init, readBME280
    bme280init(i2c)
    while True:
        temp, pressure, humid = readBME280()
        fbuff.fill(0)
        fbuff.fill(0)
        fbuff.text("BME280", 0, 0, 1)
        fbuff.text("tmp:", 0, 16, 1)
        fatntext("%.2f"%(temp), 32, 10)
        fbuff.text("prs:", 0, 36, 1)
        fatntext("%.2f"%(pressure), 32, 30)
        fbuff.text("hum:", 0, 54, 1)
        fatntext("%.2f"%(humid), 32, 50)
        oledshow()
        time.sleep(0.1)

if devname == "BNO055":
    from BNO055serial_funcs import InitBNO055, BNO055absorientation, BNO055calibstat
    InitBNO055(uart)
    while True:
        fbuff.fill(0)
        calibstat = None
        try:
            pitch, roll, orient = BNO055absorientation()
            calibstat = BNO055calibstat()
        except Exception as e:
            fbuff.text("BNO055", 0, 0, 1)
            fbuff.text(e.args[0], 0, 16, 1)
        if calibstat is not None:  
            fbuff.text("BNO055 :{0:08b}".format(calibstat), 0, 0, 1)
            fbuff.text("pitch:", 8, 16, 1)
            fatntext("%d"%int(pitch), 64, 10)
            fbuff.text("roll:", 16, 32, 1)
            fatntext("%d"%int(roll), 64, 28)
            fbuff.text("orient:", 0, 52, 1)
            fatntext("%d"%int(orient), 64, 48)
        oledshow()
        time.sleep(0.1)        

        
# read any GPS serial signals on 
import machine
uart.init(baudrate=9600)
while True:
    x = uart.readline()
    if x is not None:
        fbuff.fill(0)
        fbuff.text("GPS serial", 0, 0, 1)
        x = x.strip()
        for i in range(len(x)//16 + 1):
            fbuff.text(x[i*16:i*16+16], 0, 15+i*8, 1)
        oledshow()
        time.sleep_ms(100)
    time.sleep_ms(50)


Sent 152 lines (4654 bytes) to main.py.


In [749]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [756]:
import array, ustruct


dc = array.array("i", range(18))  # calibrations
dT1 = const(0);  dT2 = const(1);  dT3 = const(2); # these are indexes into dc
dP1 = const(3);  dP2 = const(4);  dP3 = const(5);  dP4 = const(6); dP5 = const(7); 
dP6 = const(8);  dP7 = const(9);  dP8 = const(10); dP9 = const(11); 
dH1 = const(12); dH2 = const(13); dH3 = const(14); dH4 = const(15); dH5 = const(16); dH6 = const(17); 

def bme280init(li2c):
    global i2c
    i2c = li2c
    assert i2c.readfrom_mem(0x77, 0xD0, 1) == b'\x60', "chip not found"
    
    i2c.writeto_mem(0x77, 0xF2, b'\x01')  # switch on humidity (over)sampling=1
    i2c.writeto_mem(0x77, 0xF4, b'\x27')  # switch on temp and pressure (over)sampling=1, mode=normal(continuous readings)
    i2c.writeto_mem(0x77, 0xF5, b'\x00')  # 0.5ms standby, no filtering, no-SPI

    dc[dT1], dc[dT2], dc[dT3] = ustruct.unpack("<Hhh", i2c.readfrom_mem(0x77, 0x88, 6))
    dc[dP1], dc[dP2], dc[dP3], dc[dP4], dc[dP5], dc[dP6], dc[dP7], dc[dP8], dc[dP9] = \
            ustruct.unpack("<Hhhhhhhhh", i2c.readfrom_mem(0x77, 0x8E, 18))
        
    dc[dH1] = i2c.readfrom_mem(0x77, 0xA1, 1)[0]
    dig_e1_e7 = i2c.readfrom_mem(0x77, 0xE1, 7)
    dc[dH2], dc[dH3] = ustruct.unpack("<hB", dig_e1_e7)
    e4_sign = ustruct.unpack_from("<b", dig_e1_e7, 3)[0]
    dc[dH4] = (e4_sign << 4) | (dig_e1_e7[4] & 0xF)
    e6_sign = ustruct.unpack_from("<b", dig_e1_e7, 5)[0]
    dc[dH5] = (e6_sign << 4) | (dig_e1_e7[4] >> 4)
    dc[dH6] = ustruct.unpack_from("<b", dig_e1_e7, 6)[0]


readout = bytearray(8)
tph = array.array("i", range(3))

def readBME280():
    # burst readout from 0xF7 to 0xFE, recommended by datasheet
    i2c.readfrom_mem_into(0x77, 0xF7, readout)
    raw_press = ((readout[0] << 16) | (readout[1] << 8) | readout[2]) >> 4
    raw_temp = ((readout[3] << 16) | (readout[4] << 8) | readout[5]) >> 4
    raw_hum = (readout[6] << 8) | readout[7]

    # temperature
    var1 = ((raw_temp >> 3) - (dc[dT1] << 1)) * (dc[dT2] >> 11)
    var2 = (((((raw_temp >> 4) - dc[dT1]) * ((raw_temp >> 4) - dc[dT1])) >> 12) * dc[dT3]) >> 14
    t_fine = var1 + var2
    tempout = (t_fine * 5 + 128) >> 8
    
    # pressure
    var1 = t_fine - 128000
    var2 = var1 * var1 * dc[dP6]
    var2 = var2 + ((var1 * dc[dP5]) << 17)
    var2 = var2 + (dc[dP4] << 35)
    var1 = (((var1 * var1 * dc[dP3]) >> 8) + ((var1 * dc[dP2]) << 12))
    var1 = (((1 << 47) + var1) * dc[dP1]) >> 33
    if var1 == 0:
        pressureout = 0
    else:
        p = 1048576 - raw_press
        p = (((p << 31) - var2) * 3125) // var1
        var1 = (dc[dP9] * (p >> 13) * (p >> 13)) >> 25
        var2 = (dc[dP8] * p) >> 19
        pressureout = ((p + var1 + var2) >> 8) + (dc[dP7] << 4)
    
    # humidity
    h = t_fine - 76800
    h = (((((raw_hum << 14) - (dc[dH4] << 20) - (dc[dH5] * h)) + 16384) >> 15) * (((((((h * dc[dH6]) >> 10) * (((h * dc[dH3]) >> 11) + 32768)) >> 10) + 2097152) * dc[dH2] + 8192) >> 14))
    h = h - (((((h >> 15) * (h >> 15)) >> 7) * dc[dH1]) >> 4)
    h = 0 if h < 0 else h
    h = 419430400 if h > 419430400 else h
    humidityout = h >> 12
    return (tempout/100, pressureout/256, humidityout/1024)


        
        

In [757]:
bme280init(OLED_driver.i2c)

In [766]:
temp, pressure, humid = readBME280()


In [772]:
print(chr(46))

.


In [775]:
num8x8 = b'\x00\x08\x08\x08\x08\x08\x08\x00\x00\x00\x00``\x00\x00\x00\x00@`0\x18\x0c\x06\x02\x00>\x7fIE\x7f>\x00\x00@D\x7f\x7f@@\x00\x00bsQIOF\x00\x00"cII\x7f6\x00\x00\x18\x18\x14\x16\x7f\x7f\x10\x00\'gEE}9\x00\x00>\x7fII{2\x00\x00\x03\x03y}\x07\x03\x00\x006\x7fII\x7f6\x00\x00&oII\x7f>\x00'
def fatntext(t, x, y):  # numbers only
    for s in t:
        k = max(0, min(11, ord(s) - 45))
        if k == 1:  x -= 4
        for c in num8x8[k*8:k*8+8]:
            for j in range(8):
                if (c & (1<<j)):
                    fbuff.fill_rect(x,y+j*2,2,2,1)
            x += 2
        if k == 1:  x -= 4


In [782]:
temp, pressure, humid = readBME280()
fbuff.fill(0)
fbuff.text("BME280", 0, 0, 1)
fbuff.text("tmp:", 0, 16, 1)
fatntext("%.2f"%(temp), 32, 10)
fbuff.text("prs:", 0, 36, 1)
fatntext("%.2f"%(pressure), 32, 30)
fbuff.text("hum:", 0, 54, 1)
fatntext("%.2f"%(humid), 32, 50)
oledshow()


In [751]:
print(i2c.scan())
k = i2c.readfrom_mem(0x77, 0xD0, 1)
print(k, b'\x60')

[60, 119]
b'`' b'`'


In [737]:
%sendtofile --source mlx90621new.py

Sent 126 lines (4770 bytes) to mlx90621new.py.


In [739]:
import mlx90621new

24.83673 -39.80357


In [741]:
mlx90621new.measure()
print(mlx90621new.temperatures)


[18.75223, 20.35458, 22.10349, 23.63788, 19.22498, 19.22794, 22.39536, 25.92029, 19.76028, 18.81921, 22.10397, 22.61749, 17.85779, 21.25632, 21.51877, 23.23511, 21.24747, 21.40778, 21.87399, 21.17954, 18.35764, 22.12381, 20.96909, 21.62027, 19.6044, 20.93777, 21.22958, 18.69333, 21.09058, 21.46814, 19.75803, 18.75711, 21.40778, 21.07797, 18.15628, 18.00375, 20.73462, 21.58798, 17.19711, 17.26736, 19.55396, 19.79221, 18.2619, 17.56912, 17.90182, 19.69659, 17.52167, 17.68277, 17.91833, 17.6832, 20.46457, 18.27875, 19.10852, 18.99969, 21.56375, 19.93799, 19.48215, 22.65323, 20.86014, 22.35367, 19.46313, 18.94232, 20.22464, 21.6553]


In [728]:
# looking at the MLX
print(i2c.scan(), 0x50, 0x60)

[60, 80, 81, 82, 83, 84, 85, 86, 87, 96] 80 96


In [729]:
import machine, ustruct, math

# fetch  16 bit register [0x02:readcommand, reg, addressstep=0, numberofreads=1]
# (the interface to the MLX90621 is so you can request a row or column
#  of pixels with one command call)
def readreg16(reg, signed=False):
    i2c.writeto(0x60, bytearray([0x02, reg, 0x00, 0x01]), False)
    return ustruct.unpack("<h" if signed else "<H", i2c.readfrom(0x60, 2))[0]

# read signed/unsigned 1 or 2 byte number
def readEEprom(reg, nc):
    return ustruct.unpack(nc[1], i2c.readfrom_mem(0x50, reg, nc[0]))[0]
s16 = (2, "<h")
u16 = (2, "<H")
s8 = (1, "b")
u8 = (1, "B")


# read config
config = readreg16(0x92, signed=False)
needs_initializing = not (readreg16(0x92, signed=False) & 0x0400)  # also when power on

# Read the EEPROM table (see 8.2.3, 9.5)
#    deltaAi = i2c.readfrom_mem(0x50, 0x00, 0x40)
#    Bi = i2c.readfrom_mem(0x50, 0x40, 0x40)
#    deltaalphai = i2c.readfrom_mem(0x50, 0x80, 0x40)

def initialize_settings():
    print("Initializing settings")
    # Write the oscillator trim value into the IO at address 0x93 (see 7.1, 8.2.2.2, 9.4.4) (no idea what this is)
    OSC_trim = readEEprom(0xF7, u8)
    i2c.writeto_mem(0x60, 0x04, bytearray([OSC_trim - 0xAA, OSC_trim, 0x56, 0x00]));
    
    # Write the configuration value (IO address 0x92) (see 8.2.2.1, 9.4.3)
    #   The value is either read from the EEPROM or hard coded externally
    #   Set the POR/ Brown Out (!needs_initializing) flag to “1” (bit 10 at address 0x92)
    Hz_LSB = 0b00111010  # 18bit resolution, 16 frames/second
    defaultConfig_H = 0b01000100  # bit 2 resets the Brown Out flag  
    i2c.writeto_mem(0x60, 0x03, bytearray([Hz_LSB - 0x55, Hz_LSB, defaultConfig_H - 0x55, defaultConfig_H]))

    # prove ConfigReg[5:3] == 3, which is part of a scaling factor
    assert ((readreg16(0x92, signed=False)>>4)&3) == 3

    

In [730]:
initialize_settings()

Initializing settings


In [676]:
uart.init(baudrate=115200)
print(uart)

UART(1, baudrate=115201, bits=8, parity=None, stop=1, tx=17, rx=13, rts=-1, cts=-1, timeout=0, timeout_char=1)


In [712]:
# this local tests the BNO055serial
import time
uart.write(b"\xAA\x01\x00\x06")  # request chip_id and firmware version
time.sleep_ms(20)
r = uart.read()
print(r)

b'\xbb\x06\xa0\xfb2\x0f\x08\x03'
